# Sensitivities and symbolic differentiation

The standard formulation of forward sensitivities is

 - State: $y = y(p, t)$
 - Constant or an initial state value: $p$ 
 - State derivatives: $f(y, p, t) = \frac{\mathrm{d}y}{\mathrm{d}t}$, this is the myokit.Model
 - Sensitivities: $s = \frac{\mathrm{d}y}{\mathrm{d}p}$
 - Sensitivity ODEs: $\frac{\mathrm{d}s}{\mathrm{d}t} = \frac{\mathrm{d}}{\mathrm{d}p}\frac{\mathrm{d}y}{\mathrm{d}t} = \frac{\partial f}{\partial y}s + \frac{\partial f}{\partial p}$,
    where the first equality follows from the commutativity of the time derivative and the parameter derivative (the parameter are constant in time). The second equality follows from substituting $f(y, p t)$ and writing out the total derivative w.r.t. to $p$.

The final sensitivity ODE expression assumes that there are no intermediate expressions that may implicitly depend on time, parameters or other states.

In myokit, we might also be interested in the sensitivities of an intermediate expression. Those can be straightforwadly derived from the above sensitivities

 - Intermediate: $x = x(y, p, t)$
 - Sensitivities: $\frac{\mathrm{d}x}{\mathrm{d}p} = \frac{\partial x}{\partial y}s + \frac{\partial x}{\partial p}$.

In [1]:
import os

import myokit
from myokit.formats.sbml import SBMLImporter

## Model: Bacterial growth under drug exposure

In this model, we assume that the total bacterial population is composed of a wild type strain $b_w$ and a mutant strain $b_m$
\begin{equation}
    b = b_w + b_m.
\end{equation}

Here $b$, $b_w$ and $b_m$ is the bacterial count. The growth of the wild type under exposure to the drug is modelled by an exponential growth $\lambda $ and a drug-induced killing $\kappa_w $ as well as mutation rate $\mu $
\begin{equation}
    \frac{\text{d}b_w}{\text{d}t} =
        \lambda b_w
        - \kappa _w c b_w
        - \mu c b_w.
\end{equation}

The mutant population is resistant to the drug and experiences a Hill-like killing with maximal killing rate $\kappa _m$ and EC50 $c_{50}$
\begin{equation}
    \frac{\text{d}b_m}{\text{d}t} =
        \lambda b_m
        - \kappa _m\frac{c}{c - c_{50}}b_m
        + \mu c b_w.
\end{equation}

The drug is administered directly to the bacterial environment

\begin{equation}
    \frac{\text{d}a}{\text{d}t} = -ka + r_{\text{dose}}(t), \quad \mathrm{and} \quad c = \frac{a}{v},
\end{equation}

where $a$ is the amount of the drug, $k$ is the elimination rate and $r_{\text{dose}}$ dose rate. $c$ is the drug concentration and $v$ is the effective volume of distribution of the drug.

In [2]:
directory = os.getcwd()
path = os.path.join(directory, 'models/simple_kill_and_resistance_model.xml')
model = SBMLImporter().model(path)
print(model.code())

[[model]]
name: adaptive_resistance
# Initial values
myokit.bacterial_count_adapted     = 1
myokit.bacterial_count_susceptible = 1
central.drug_amount                = 0

[central]
dot(drug_amount) = -(size * myokit.elimination_rate * drug_concentration)
    in [kg (1e-06)]
drug_concentration = drug_amount / size
    in [g/m^3]
size = 1
    in [L]

[myokit]
dot(bacterial_count_adapted) = (growth_rate - kappa_adapted * (central.drug_concentration^gamma / (central.drug_concentration^gamma + concentration_e50_adapted^gamma))) * bacterial_count_adapted + mutation_rate * central.drug_concentration * bacterial_count_susceptible
    in [1/m^3 (1000)]
dot(bacterial_count_susceptible) = (growth_rate - (kappa_susceptible * central.drug_concentration + mutation_rate * central.drug_concentration)) * bacterial_count_susceptible
    in [1/m^3 (1000)]
concentration_e50_adapted = 1
    in [g/m^3]
elimination_rate = 1
    in [S/F (0.0002777777777777778)]
gamma = 1
    in [1]
growth_rate = 1
    in [S/F

## Compute sensitivities analytically

The quantity that's typically measured in experiments is the total bacterial count $b$. So, we are interested in the sensitivities of $b$.

In this case, $b$ does not explicitly depend on any parameters $\frac{\partial b}{\partial p}=0$, and the depends linearly on the states $\frac{\partial b}{\partial b_w} = \frac{\partial b}{\partial b_w} = 1$. Thus, the sensitivities to all parameters are given by
\begin{equation}
    \frac{\mathrm{d}b}{\mathrm{d}p} = 
        \frac{\mathrm{d}b_w}{\mathrm{d}p} + \frac{\mathrm{d}b_m}{\mathrm{d}p}.
\end{equation}

Without loss of generality, the initial conditions of the sensitivity ODEs are $s(t=0) = 1$ for the sensitivity of states w.r.t. their own initial conditions of states and $s(t=0) = 0$ else. As a result, $\dot{s}=0$ implies $s \equiv 1$ for the initial conditions of the same states and $s \equiv 0$ for all other parameters.

### Sensitivities to parameters that are not initial conditions:
For all subsequent sensitivity ODEs, the initial conditions are $s(t=0) = 0$.

#### Sensitivity to $\lambda$:
For ease of notation we use a superscript to indicate the subgroup and a subscript to indicate the parameter
\begin{equation}
    \frac{\mathrm{d}s^{w}_{\lambda}}{\mathrm{d}t} = 
        \left( 
            \lambda - \kappa _w c - \mu c
        \right) s^{w}_{\lambda} + b_w
    \quad \mathrm{and} \quad
    \frac{\mathrm{d}s^{m}_{\lambda}}{\mathrm{d}t} = 
        \left( 
            \lambda - \kappa _m\frac{c}{c - c_{50}}
        \right) s^{m}_{\lambda} + \mu c s^{w}_{\lambda} + b_m.
\end{equation}

#### Sensitivity to $\kappa _w$:
\begin{equation}
    \frac{\mathrm{d}s^{w}_{\kappa _w}}{\mathrm{d}t} = 
        \left( 
            \lambda - \kappa _w c - \mu c
        \right) s^{w}_{\kappa _w} + c b_w
    \quad \mathrm{and} \quad
    \frac{\mathrm{d}s^{m}_{\kappa _w}}{\mathrm{d}t} = 
        \mu c s^{w}_{\kappa _w}.
\end{equation}

#### Sensitivity to $\mu$:
\begin{equation}
    \frac{\mathrm{d}s^{w}_{\mu}}{\mathrm{d}t} = 
        \left( 
            \lambda - \kappa _w c - \mu c
        \right) s^{w}_{\mu} + c b_w
    \quad \mathrm{and} \quad
    \frac{\mathrm{d}s^{m}_{\mu}}{\mathrm{d}t} = 
        \left( 
            \lambda - \kappa _m\frac{c}{c - c_{50}}
        \right) s^{m}_{\mu} + \mu c s^{w}_{\mu} + c b_w.
\end{equation}

#### Sensitivity to $\kappa _m$:
\begin{equation}
    \frac{\mathrm{d}s^{w}_{\kappa _m}}{\mathrm{d}t} = 
        0
    \quad \mathrm{and} \quad
    \frac{\mathrm{d}s^{m}_{\kappa _m}}{\mathrm{d}t} = 
        \left( 
            \lambda - \kappa _m\frac{c}{c - c_{50}}
        \right) s^{m}_{\kappa _m} + \frac{c}{c - c_{50}}b_w.
\end{equation}

#### Sensitivity to $c_{50}$:
\begin{equation}
    \frac{\mathrm{d}s^{w}_{c_{50}}}{\mathrm{d}t} = 
        0
    \quad \mathrm{and} \quad
    \frac{\mathrm{d}s^{m}_{c_{50}}}{\mathrm{d}t} = 
        \left( 
            \lambda - \kappa _m\frac{c}{c - c_{50}}
        \right) s^{m}_{c_{50}} + \kappa_m\frac{c}{(c - c_{50})^2}b_w.
\end{equation}

#### Sensitivity to $v$:
The sensitivity to $v$ is interesting, because $b$ depends on $v$ implicitly through either $b_w$ or $b_m$ and $c$. The sensitivity might be abstractly expressed as
\begin{equation}
    \frac{\mathrm{d}s^{i}_v}{\mathrm{d}t} = \frac{\mathrm{d}}{\mathrm{d}v} f(b_i, c, v) =  \frac{\partial f}{\partial b_i}s^{i}_v + \frac{\partial f}{\partial c}\frac{\mathrm{d}c}{\mathrm{d}v} + \frac{\partial f}{\partial v}.
\end{equation}
In this case, $\frac{\partial f}{\partial v} = 0$. Thus, the sensitivities are given by
\begin{equation}
    \frac{\mathrm{d}s^{w}_{v}}{\mathrm{d}t} = 
        \left( 
            \lambda - \kappa _w c - \mu c
        \right) s^{w}_{v} +
        \left( 
            \kappa _w b_w + \mu b_w
        \right) \frac{a}{v^2}
    \quad \mathrm{and} \quad
    \frac{\mathrm{d}s^{m}_{v}}{\mathrm{d}t} = 
        \left( 
            \lambda - \kappa _m\frac{c}{c - c_{50}}
        \right) s^{m}_{v} + \mu c s^{w}_{v} -
        \left( 
            \mu b_w - \kappa _m\frac{1}{c - c_{50}} b_m + \kappa _m\frac{c}{(c - c_{50})^2} b_m
        \right) \frac{a}{v^2}
\end{equation}

#### Sensitivity to $k$:
The sensitivity to $k$ is even deeper rooted than the sensitivity to $v$. The sensitivity might be abstractly expressed as
\begin{equation}
    \frac{\mathrm{d}s^{i}_k}{\mathrm{d}t} = \frac{\mathrm{d}}{\mathrm{d}k} f(b_i, c(a, k), k) =  \frac{\partial f}{\partial b_i}s^{i}_k + \frac{\partial f}{\partial c}\left( \frac{\partial c}{\partial a}\frac{\mathrm{d}a}{\mathrm{d}k} + \frac{\partial c}{\partial k}\right) + \frac{\partial f}{\partial k}.
\end{equation}
In this case, $\frac{\partial c}{\partial k} = \frac{\partial f}{\partial k} = 0$. Thus, the sensitivities are given by
\begin{equation}
    \frac{\mathrm{d}s^{w}_{k}}{\mathrm{d}t} = 
        \left( 
            \lambda - \kappa _w c - \mu c
        \right) s^{w}_{k} -
        \left( 
            \kappa _w b_w + \mu b_w
        \right) \frac{1}{v}\frac{\mathrm{d}a}{\mathrm{d}k}
    \quad \mathrm{and} \quad
    \frac{\mathrm{d}s^{m}_{k}}{\mathrm{d}t} = 
        \left( 
            \lambda - \kappa _m\frac{c}{c - c_{50}}
        \right) s^{m}_{v} + \mu c s^{w}_{k} +
        \left( 
            \mu b_w - \kappa _m\frac{1}{c - c_{50}} b_m + \kappa _m\frac{c}{(c - c_{50})^2} b_m
        \right) \frac{1}{v}\frac{\mathrm{d}a}{\mathrm{d}k}
\end{equation}

#### Sensitivity of $a$ to $k$:
The sensitivity of $a$ to $k$ has to be computed with an additional forward sensitivity ODE
\begin{equation}
    \frac{\mathrm{d}s^{a}_{k}}{\mathrm{d}t} = 
        -k s^{a}_{k} - a,
\end{equation}

where $s^{a}_{k} := \frac{\mathrm{d}a}{\mathrm{d}k}$.

### Sensitivities to initial conditions:
For all subsequent sensitivity ODEs, the initial conditions are $s(t=0) = 1$ for the sensitivities of states to their own initial conditions and $s(t=0) = 0$ otherwise.

#### Sensitivity to $b_{w,0}$:
\begin{equation}
    \frac{\mathrm{d}s^{w}_{b_{w,0}}}{\mathrm{d}t} = 
        \left( 
            \lambda - \kappa _w c - \mu c
        \right) s^{w}_{b_{w,0}}
    \quad \mathrm{and} \quad
    \frac{\mathrm{d}s^{m}_{\lambda}}{\mathrm{d}t} = 
        \left( 
            \lambda - \kappa _m\frac{c}{c - c_{50}}
        \right) s^{m}_{b_{w,0}} + \mu c s^{w}_{b_{w,0}}.
\end{equation}

#### Sensitivity to $b_{m, 0}$:
\begin{equation}
    \frac{\mathrm{d}s^{w}_{b_{m, 0}}}{\mathrm{d}t} = 
        0
    \quad \mathrm{and} \quad
    \frac{\mathrm{d}s^{m}_{b_{m, 0}}}{\mathrm{d}t} = 
        \left( 
            \lambda - \kappa _m\frac{c}{c - c_{50}}
        \right) s^{m}_{b_{m, 0}}.
\end{equation}

#### Sensitivity to $a_0$:
\begin{equation}
    \frac{\mathrm{d}s^{w}_{a_0}}{\mathrm{d}t} = 
        \left( 
            \lambda - \kappa _w c - \mu c
        \right) s^{w}_{a_0} -
        \left( 
            \kappa _w b_w + \mu b_w
        \right) \frac{1}{v}s^{a}_{a_0}
    \quad \mathrm{and} \quad
    \frac{\mathrm{d}s^{m}_{a_0}}{\mathrm{d}t} = 
        \left( 
            \lambda - \kappa _m\frac{c}{c - c_{50}}
        \right) s^{m}_{a_0} + \mu c s^{w}_{a_0} +
        \left( 
            \mu b_w - \kappa _m\frac{1}{c - c_{50}} b_m + \kappa _m\frac{c}{(c - c_{50})^2} b_m
        \right) \frac{1}{v}s^{a}_{a_0}
    \quad \mathrm{and} \quad
        \frac{\mathrm{d}s^{a}_{a_0}}{\mathrm{d}t} = 
            -k s^{a}_{a_0}.
\end{equation}

## Algorithm to derive sensitivity ODEs

From this toy example, we can see that in order to derive the sensitivities of either states or intermediate expressions, we need to keep track of explicit and implicit dependencies of the right hand side of any expression of interest. In order for a state or intermediate expressionto have a non-vanishing sensitivity to a model parameter, it has to either explicitly depend on that parameter or implicitly depend on the parameter through (possibly multiple) states or intermediate expressions. If a state is sensitive to a parameter, the above construction leads to an ODE for the sensitivity. If an intermediate expression is sensitive to a parameter, we obtain an algebraic expression for its sensitivity. It's important to note that forward sensitivities can lead to coupled systems of sensitivity ODEs, which might justify a more explicit notation

\begin{equation}
    \frac{\mathrm{d}s^i}{\mathrm{d}t} = \sum _j \frac{\partial f^i}{\partial y^j}s^j + \sum _k \frac{\partial f^i}{\partial x^k}\frac{\mathrm{d}x^k}{\mathrm{d}p} + \frac{\partial f^i}{\partial p}
\end{equation}
\begin{equation}
    \frac{\mathrm{d}x^i}{\mathrm{d}p} = \sum _j \frac{\partial f^i}{\partial y^j}s^j + \sum _{k\neq i} \frac{\partial f^i}{\partial x^k}\frac{\mathrm{d}x^k}{\mathrm{d}p} + \frac{\partial f^i}{\partial p},
\end{equation}

where the $j$ runs over all states and $k$ runs over all intermediate expressions. $f^i$ is the right hand side of either the state or intermediate expression $y^i$ or $x^i$.

This puts us in a position to formulate a simple algorithm to construct the system of sensitivity ODEs:

1. Select state / intermediate expression of interest. From now on referred to as Y.
2. Select the parameter of interest. From now on P.
3. Get RHS of Y.
4. Construct a tree of partial derivatives (see subroutine).
5. Identify relevant branches of the tree, by starting at the deepest level. If there is no explicit dependence on P, level can be ignored and we repeat the check at the previous level. If there is an explicit dependence, the entire branch is relevant for the constructions.
6. Construct system of ODEs by summing the expressions at each level from the relevant branches.

Subroutine: Construct a tree of partial derivatives:

1. Initialise first node of tree at Y
2. Create nodes of the tree by adding a node for each state $y$, intermediate expression $x$ and the parameter P.
3. Fill the nodes with the partial derivatives multiplied by the total derivate of x, y and P w.r.t. P. Delete those nodes with vanishing derivatives.
4. For each level 1 node derived from a state or a intermediate expression, repeat 2. and 3..
5. If a state or intermediate expression appears a second time in the tree, references the first appearance.
6. Terminate when no new states or intermediate expressions appear.

In [3]:
# Get partials of total_bacterial_count
derivative_tree = {}
unexplored = set()
parameter = model.get('myokit.growth_rate').lhs()
var = model.get('myokit.total_bacterial_count')
lhs = var.lhs()
rhs = var.rhs()
derivative_tree['myokit.total_bacterial_count'] = {
    'is state': lhs.is_derivative(),
    'sensitivity': None,
    'partial derivatives': {
        'parameters': {parameter.__str__(): rhs.diff(parameter)},
        'not constant': {a.__str__(): rhs.diff(a) for a in rhs.references() if not a.is_constant()}
    }
}

# Add non-constant parameters to list of unexplored dependencies, if there not already in the derivative tree
explored = derivative_tree.keys()
for dependency in derivative_tree['myokit.total_bacterial_count']['partial derivatives']['not constant'].keys():
    # Skip, if already explored
    if dependency in explored:
        continue
    # Add depency to unexplored list
    unexplored.add(dependency)

print(derivative_tree)
print(unexplored)

{'myokit.total_bacterial_count': {'is state': False, 'sensitivity': None, 'partial derivatives': {'parameters': {'myokit.growth_rate': myokit.Expression[0 [s/m^3 (4e+06)]]}, 'not constant': {'myokit.bacterial_count_susceptible': myokit.Expression[1], 'myokit.bacterial_count_adapted': myokit.Expression[1]}}}}
{'myokit.bacterial_count_susceptible', 'myokit.bacterial_count_adapted'}


In [4]:
# Get partials for next dependency
name = unexplored.pop()
var = model.get(name)
lhs = var.lhs()
rhs = var.rhs()
derivative_tree[name] = {
    'is state': lhs.is_derivative(),
    'sensitivity': None,
    'partial derivatives': {
        'parameters': {parameter.__str__(): rhs.diff(parameter)},
        'not constant': {a.__str__(): rhs.diff(a) for a in rhs.references() if not a.is_constant()}
    }
}

# Add non-constant parameters to list of unexplored dependencies, if there not already in the derivative tree
explored = derivative_tree.keys()
for dependency in derivative_tree[name]['partial derivatives']['not constant'].keys():
    # Skip, if already explored
    if dependency in explored:
        continue
    # Add depency to unexplored list
    unexplored.add(dependency)

print(derivative_tree.keys())
print(derivative_tree[name]['partial derivatives']['parameters'])
print(derivative_tree[name]['partial derivatives']['not constant'].keys())
print(unexplored)

dict_keys(['myokit.total_bacterial_count', 'myokit.bacterial_count_susceptible'])
{'myokit.growth_rate': myokit.Expression[1 * myokit.bacterial_count_susceptible]}
dict_keys(['myokit.bacterial_count_susceptible', 'central.drug_concentration'])
{'myokit.bacterial_count_adapted', 'central.drug_concentration'}


In [5]:
# Get partials for next dependency
name = unexplored.pop()
var = model.get(name)
lhs = var.lhs()
rhs = var.rhs()
derivative_tree[name] = {
    'is state': lhs.is_derivative(),
    'sensitivity': None,
    'partial derivatives': {
        'parameters': {parameter.__str__(): rhs.diff(parameter)},
        'not constant': {a.__str__(): rhs.diff(a) for a in rhs.references() if not a.is_constant()}
    }
}

# Add non-constant parameters to list of unexplored dependencies, if there not already in the derivative tree
explored = derivative_tree.keys()
for dependency in derivative_tree[name]['partial derivatives']['not constant'].keys():
    # Skip, if already explored
    if dependency in explored:
        continue
    # Add depency to unexplored list
    unexplored.add(dependency)

print(derivative_tree.keys())
print(derivative_tree[name]['partial derivatives']['parameters'])
print(derivative_tree[name]['partial derivatives']['not constant'].keys())
print(unexplored)

dict_keys(['myokit.total_bacterial_count', 'myokit.bacterial_count_susceptible', 'myokit.bacterial_count_adapted'])
{'myokit.growth_rate': myokit.Expression[1 * myokit.bacterial_count_adapted]}
dict_keys(['myokit.bacterial_count_susceptible', 'central.drug_concentration', 'myokit.bacterial_count_adapted'])
{'central.drug_concentration'}


In [6]:
# Get partials for next dependency
name = unexplored.pop()
var = model.get(name)
lhs = var.lhs()
rhs = var.rhs()
derivative_tree[name] = {
    'is state': lhs.is_derivative(),
    'sensitivity': None,
    'partial derivatives': {
        'parameters': {parameter.__str__(): rhs.diff(parameter)},
        'not constant': {a.__str__(): rhs.diff(a) for a in rhs.references() if not a.is_constant()}
    }
}

# Add non-constant parameters to list of unexplored dependencies, if there not already in the derivative tree
explored = derivative_tree.keys()
for dependency in derivative_tree[name]['partial derivatives']['not constant'].keys():
    # Skip, if already explored
    if dependency in explored:
        continue
    # Add depency to unexplored list
    unexplored.add(dependency)

print(derivative_tree.keys())
print(derivative_tree[name]['partial derivatives']['parameters'])
print(derivative_tree[name]['partial derivatives']['not constant'].keys())
print(unexplored)

dict_keys(['myokit.total_bacterial_count', 'myokit.bacterial_count_susceptible', 'myokit.bacterial_count_adapted', 'central.drug_concentration'])
{'myokit.growth_rate': myokit.Expression[0 [g*s/m^3 (3600)]]}
dict_keys(['central.drug_amount'])
{'central.drug_amount'}


In [7]:
# Get partials for next dependency
name = unexplored.pop()
var = model.get(name)
lhs = var.lhs()
rhs = var.rhs()
derivative_tree[name] = {
    'is state': lhs.is_derivative(),
    'sensitivity': None,
    'partial derivatives': {
        'parameters': {parameter.__str__(): rhs.diff(parameter)},
        'not constant': {a.__str__(): rhs.diff(a) for a in rhs.references() if not a.is_constant()}
    }
}

# Add non-constant parameters to list of unexplored dependencies, if there not already in the derivative tree
explored = derivative_tree.keys()
for dependency in derivative_tree[name]['partial derivatives']['not constant'].keys():
    # Skip, if already explored
    if dependency in explored:
        continue
    # Add depency to unexplored list
    unexplored.add(dependency)

print(derivative_tree.keys())
print(derivative_tree[name]['partial derivatives']['parameters'])
print(derivative_tree[name]['partial derivatives']['not constant'].keys())
print(unexplored)

dict_keys(['myokit.total_bacterial_count', 'myokit.bacterial_count_susceptible', 'myokit.bacterial_count_adapted', 'central.drug_concentration', 'central.drug_amount'])
{'myokit.growth_rate': myokit.Expression[0 [kg (1e-06)]]}
dict_keys(['central.drug_concentration'])
set()


### Construct sensitivity equations from derivative tree

In [9]:
# Add a sensitivity compartment to the model
if not model.has_component('sensitivities'):
    comp = model.add_component_allow_renaming('sensitivities')

for name, details in derivative_tree.items():
    # Get/add sensitivity variable to model
    try:
        var = comp.get(details['sensitivity'])
    except AttributeError:
        var = comp.add_variable_allow_renaming(
            'd%s_d%s' % (name.replace('.', '_'), str(parameter).replace('.', '_'))
        )
        details['sensitivity'] = var

print(model.code())

[[model]]
name: adaptive_resistance
# Initial values
myokit.bacterial_count_adapted     = 1
myokit.bacterial_count_susceptible = 1
central.drug_amount                = 0

[central]
dot(drug_amount) = -(size * myokit.elimination_rate * drug_concentration)
    in [kg (1e-06)]
drug_concentration = drug_amount / size
    in [g/m^3]
size = 1
    in [L]

[myokit]
dot(bacterial_count_adapted) = (growth_rate - kappa_adapted * (central.drug_concentration^gamma / (central.drug_concentration^gamma + concentration_e50_adapted^gamma))) * bacterial_count_adapted + mutation_rate * central.drug_concentration * bacterial_count_susceptible
    in [1/m^3 (1000)]
dot(bacterial_count_susceptible) = (growth_rate - (kappa_susceptible * central.drug_concentration + mutation_rate * central.drug_concentration)) * bacterial_count_susceptible
    in [1/m^3 (1000)]
concentration_e50_adapted = 1
    in [g/m^3]
elimination_rate = 1
    in [S/F (0.0002777777777777778)]
gamma = 1
    in [1]
growth_rate = 1
    in [S/F

### TODO:
1. Extend tree to contain flag, whether rhs is explicitly dependent on parameter.
2. Extend routine that removes all entries in the tree that would evaluate to zero.
3. The remaining expressions will be relevant, and can then be used to construct the sensitivities.